In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/training/

Mounted at /content/drive
 Ass2_NLP.ipynb		       news-commentary-v9.de-en.en
'new 1.py'		       news-commentary-v9.fr-en.en
 news-commentary-v9.cs-en.cs   news-commentary-v9.fr-en.fr
 news-commentary-v9.cs-en.en   news-commentary-v9.ru-en.en
 news-commentary-v9.de-en.de   news-commentary-v9.ru-en.ru


In [2]:
with open('/content/drive/MyDrive/training/news-commentary-v9.cs-en.en','rb') as f:
  train_en=f.readlines()
with open('/content/drive/MyDrive/training/news-commentary-v9.cs-en.cs','rb') as f:
  train_fr=f.readlines()
print(len(train_en),len(train_fr))
max=0
for j,i in enumerate(train_en+train_fr):
  if len(i.decode('utf-8').split())>max:
    max=len(i.decode('utf-8').split())
print(max)
MAX_LENGTH=max
#Put max length as the maximum length of the sentence in the pair

146549 146549
164


In [3]:
%matplotlib inline
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):    
    s=s.lower().strip()
    s=s.decode('utf-8')
    s = unicodeToAscii(s)
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
pairs = [[normalizeString(i),normalizeString(j)] for i,j in zip(train_fr,train_en)]
len(pairs)

146549

In [9]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5)
for i,j in kf.split(pairs):
  print(len(i),len(j))
 

117239 29310
117239 29310
117239 29310
117239 29310
117240 29309


In [10]:
#input_lang = Lang('de')
#output_lang = Lang('eng')
#print("Read %s sentence pairs" % len(pairs))
#pairs = filterPairs(pairs)
#print("Trimmed to %s sentence pairs" % len(pairs))
#num_test = int(len(pairs)/5)
#print(num_test)
#l1=pairs[:num_test]
#l2=pairs[num_test:num_test*2]
#l3=pairs[num_test*2:num_test*3]
#l4=pairs[num_test*3:num_test*4]
#l5=pairs[num_test*4:]
def manage_test_data(test_pairs):
  set_test_eng = set([sent_eng for sent_eng, _ in test_pairs])

  test_pair_dict = {}
  for sent_eng, sent_fre in pairs:
      if sent_eng not in set_test_eng:
          continue 
      elif sent_eng not in test_pair_dict:
          test_pair_dict[sent_eng] = set([sent_fre])
      else:
          test_pair_dict[sent_eng].add(sent_fre)
  return [(sent_eng, list(test_pair_dict[sent_eng])) for sent_eng in test_pair_dict]

#test_pairs = massage_test_data(pairs[161030:])
#train_pairs= pairs[:161030]

#print("Number of test pairs:", len(test_pairs))
#print("Number of train pairs:", len(train_pairs))

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters,train_pairs,input_lang,output_lang, print_every=1000, plot_every=100, learning_rate=0.01):
    #train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang,
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print('in train iters',len(train_pairs))
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    #print(training_pairs[0])
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

In [13]:
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward
        
        #Hypermater: w or w/o normalization
        #With normalization
        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
        
        #Without normalization
        #return self.logp / 1 + alpha * reward
    
    def __lt__(self, other):
        return self.eval() < other.eval()
    
    
from queue import PriorityQueue

def evaluate_beam_search(encoder, decoder, sentence, max_length=MAX_LENGTH, beam_size=2):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        # Number of sentence to generate
        endnodes = []
        number_required = 1
        
        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1
        
        # start beam search
        while True:
            # give up when decoding takes too long
            
            #Hyperparameter number of iterations: Chnaged this to maximum 3000
            
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue
            #elif n.leng > max_length:
            #    continue
            
            #Hyperparameter max_length: Uncomment to use it
            
            # decode for one step using decoder
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_size)
            nextnodes = []

            for new_k in range(beam_size):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                
            # increase qsize
            qsize += len(nextnodes) - 1
            
        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(number_required)]

        _, n = endnodes[0]
        utterance = []
        utterance.append(output_lang.index2word[n.wordid.item()])
        
        # back trace
        while n.prevNode != None:
            n = n.prevNode
            utterance.append(output_lang.index2word[n.wordid.item()])

        utterance = utterance[::-1]
            
    return utterance, None

**Have changed three hypermaters above: Normalizaiton, max length and number of iteration (stopping criteria) which is mentioned
below each lines of the specified hyperparameters**


In [14]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [15]:
from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder, test_pairs):
    references, candidates = [], []    
    for sents_src, sents_tgt in test_pairs:      
      tgt_words    = [sent_tgt.split(' ') for sent_tgt in sents_tgt]      
      src_words, _ = evaluate(encoder, decoder, sents_src)      
      references.append(tgt_words)
      candidates.append(src_words)      
    score1 = corpus_bleu(references, candidates,weights=(1,0,0,0))
    score2 = corpus_bleu(references, candidates,weights=(0,1,0,0))
    score3 = corpus_bleu(references, candidates,weights=(0,0,1,0))
    return score1,score2,score3

In [16]:
from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu_beam_search(encoder, decoder, beam_size,test_pairs):
    references, candidates = [], []
    for sent_eng, sents_fre in test_pairs:
        sents_fre = [sent_fre.split(' ') for sent_fre in sents_fre]
        output_words, _ = evaluate_beam_search(encoder, decoder, sent_eng, beam_size=beam_size)
        references.append(sents_fre)
        candidates.append(output_words)
    score1 = corpus_bleu(references, candidates,weights=(1,0,0,0))
    score2 = corpus_bleu(references, candidates,weights=(0,1,0,0))
    score3 = corpus_bleu(references, candidates,weights=(0,0,1,0))
    return score1,score2,score3

Training and Evaluating

**For BLEU 1,2,3 for CS-EN (for any language pair) dataset and also the beam search value (2a i,ii and iii)**

In [17]:
BLEU_1,BLEU_2,BLEU_3=[],[],[]
i=1
input_lang = Lang('cs')
output_lang = Lang('eng')
for train_index,test_index in kf.split(pairs):
  train_pairs=[pairs[i] for i in train_index]
  test_pairs=[pairs[i] for i in test_index]
  test_pairs = manage_test_data(test_pairs) 
  print("Read %s %s sentence pairs" % (len(train_pairs),len(test_pairs)))
  print("Counting words...")
  for pair in train_pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  hidden_size = 256
  encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
  attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
  trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
  b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
  print('BLEUs :',b1,b2,b3)
  BLEU_1.append(b1)
  BLEU_2.append(b2)
  BLEU_3.append(b3)
  if i==5:
    print('BLEU1: %s BLEU2: %s BLEU3: %s' % (sum(BLEU_1)/5,sum(BLEU_2)/5,sum(BLEU_3)/5))
    beam1,beam2,beam3 = evaluateBleu_beam_search(encoder1,attn_decoder1,3,test_pairs)
    print('beam search :', beam1,beam2,beam3)
    beam1,beam2,beam3 = evaluateBleu_beam_search(encoder1,attn_decoder1,5,test_pairs)
    print('beam search :', beam1,beam2,beam3)
  i=i+1

Read 117239 29234 sentence pairs
Counting words...
cs 116947
eng 38817
in train iters 117239
1m 45s (- 7m 0s) (1000 20%) 6.0813
3m 24s (- 5m 6s) (2000 40%) 5.7333
5m 6s (- 3m 24s) (3000 60%) 5.6391
6m 45s (- 1m 41s) (4000 80%) 5.4398
8m 21s (- 0m 0s) (5000 100%) 5.2935
BLEUs : 0.029041338278894106 0.0008888406185859056 1.751856527097693e-05
Read 117239 29196 sentence pairs
Counting words...
cs 129214
eng 41900
in train iters 117239
1m 48s (- 7m 13s) (1000 20%) 6.0648
3m 34s (- 5m 21s) (2000 40%) 5.7216
5m 18s (- 3m 32s) (3000 60%) 5.5346
7m 8s (- 1m 47s) (4000 80%) 5.4977
8m 54s (- 0m 0s) (5000 100%) 5.2886


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.00922011681599747 0.00011643151284046229 1.0
Read 117239 29250 sentence pairs
Counting words...
cs 129214
eng 41900
in train iters 117239
1m 49s (- 7m 16s) (1000 20%) 6.0903
3m 35s (- 5m 23s) (2000 40%) 5.7470
5m 23s (- 3m 35s) (3000 60%) 5.4817
7m 9s (- 1m 47s) (4000 80%) 5.4232
8m 52s (- 0m 0s) (5000 100%) 5.2514
BLEUs : 0.09036375325359115 0.004202524440582876 0.00012848237862808016
Read 117239 29238 sentence pairs
Counting words...
cs 129214
eng 41900
in train iters 117239
1m 46s (- 7m 7s) (1000 20%) 6.1137
3m 33s (- 5m 19s) (2000 40%) 5.7798
5m 17s (- 3m 31s) (3000 60%) 5.4224
7m 7s (- 1m 46s) (4000 80%) 5.5280
8m 54s (- 0m 0s) (5000 100%) 5.3881
BLEUs : 0.12363835147078203 0.003177003180670226 0.00010487052305713653
Read 117240 29217 sentence pairs
Counting words...
cs 129214
eng 41900
in train iters 117240
1m 48s (- 7m 15s) (1000 20%) 6.2172
3m 38s (- 5m 27s) (2000 40%) 5.7610
5m 24s (- 3m 36s) (3000 60%) 5.5213
7m 13s (- 1m 48s) (4000 80%) 5.5198
8m 58s (- 0m 0s) (500

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.08617234666927065 0.004377521362507757 6.410658987900934e-06
BLEU1: 0.06768718129770708 BLEU2: 0.0025524642230374452 BLEU3: 0.2000514564251888
beam search : 0.09717086943341026 0.0055640752107671805 1.7104072151396475e-05
beam search : 0.09542567936583202 0.005103040312016294 2.730475465114581e-05
